
# Python 3 - Raspar dados não estruturados com BeautifulSoup

<h4>Por <a href="https://www.twitter.com/tbkaas">Tommy Kaas</a> e <a href="https://www.twitter.com/nmulvad">Nils Mulvad</a>, Kaas & Mulvad</h4>

Este notebook acompanha a sessão "Python 3" para recuperar **dados não estruturados** da Internet.

Na última sessão, usamos a biblioteca `requests` para recuperar **dados estruturados** da internet. Também tentamos engenharia reversa de sites para encontrar uma API.

Trabalhamos com o  **JSON** formato, que extraímos do nosso `response` objeto usando o `response.json()` método.

Porém, quando chamamos URLs que nos deram **JSON**, agora recuperaremos dados para um site sem essas opções e usaremos uma nova biblioteca para encontrar e extrair os snippets necessários, na grande caixa bagunçada de HTML e CSS .

---

Entra o `BeautifulSoup`.
![](images/unstructured_data.jpg)

---



 
---

![](images\soup_doc.jpg)

---

A documentação é encontrada aqui: `https://www.crummy.com/software/BeautifulSoup/bs4/doc/`.

O primeiro passo é importar a ferramenta. `BeautifulSoup` faz parte de um pacote chamado `bs4`. Uma vez que não precisamos de todo `bs4`, mas apenas `BeautifulSoup`, nós apenas importamos esta parte.

Isto se faz do seguinte modo:

`from bs4 import BeautifulSoup`

Experimente abaixo - e importe `requests` também.

In [ ]:
# importe bs4 aqui - e importe também requests. Vamos precisar dos dois ...
import requests
from bs4 import BeautifulSoup

O primeiro passo é abrir um site e obter dados para o BeautifulSoup.
Neste exemplo, usaremos um site com informações sobre jogadoras de futebol. Mais específico: queremos raspar 12 páginas com informações de todos os jogadoras da Copa do Mundo Feminina de 2019. Os dados incluem nome, país, data de nascimento, altura e posição.

A primeira das 12 páginas está aqui:
https://www.worldfootball.net/players_list/frauen-wm-2019-frankreich/nach-mannschaft/1/

Você deve abrir e verificar. É sempre uma boa idéia ser minucioso ao pesquisar o site e as páginas que você deseja raspar.

Vamos obter dados da primeira página. Vamos usar um verdadeiro `"User-Agent"`, porque não vamos gritar muito alto, que somos robôs.


In [ ]:
# defina uma variável para o URL. Você receberá uma ajudinha agora ...
url = "https://www.worldfootball.net/players_list/frauen-wm-2019-frankreich/nach-mannschaft/1/"

# crie um header
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36'
}

# retrive data to an object called r. Don't forget the header.
r = requests.get(url, headers=header)

# extraímos o código fonte da página em um objeto chamado html.
html = r.text

# agora vamos enviar o código fonte através do BeautifulSoup e carregar o produto em um objeto chamado soup
soup = BeautifulSoup(html, "html.parser")

#vamos imprimi-lo.
#print(soup)

A primeira parte dos scripts é apenas uma repetição do que fizemos na última sessão (obtendo dados estruturados).

Nós então usamos `.json()` para processar a resposta que recebemos do servidor, mas agora (linha 13 no script acima) usamos `.text`.

O motivo é que o servidor não retorna  **JSON**-data, mas **HTML**. Se tentássemos usar `.json()`, receberíamos um erro (uma *exceção*).

Basicamente, você pode dizer que `r.text` é usado se precisarmos obter o que corresponde ao código-fonte de um site (ou seja, pressionar CTRL + U no Chrome), ou seja, o HTML puro.

Na linha 16, carregamos nosso *código-fonte*, html, no `BeautifulSoup`. É habitual entre usuários de `BeautifulSoup` chamar o objeto no qual o HTML está carregado de `soup` - assim como, por exemplo, também está entre os usuários de `requests` chamar a resposta do servidor de `r`. Então, faremos isso aqui também.

Novo também é o segundo argumento em `BeautifulSoup`: `"html.parser"`.

Este argumento diz `BeautifulSoup` qual *analisador* (parser) precisa rastrear o HTML e, entre outras coisas, corrigir erros e omissões.

`html.parser` está embutido no Python 3. (O Python vem com *baterias incluídas*). Mas também existem outros analisadores, incluindo um chamado` lxml`, que é mais rápido e mais adequado para, por exemplo, analisar dados **XML**.

Contudo, `lxml` não vem com o pacote desde o início e é um programa separado que historicamente tem sido um pouco difícil de instalar no Windows.

Portanto, neste processo, usamos o que acompanha o Python, `"html.parser"`.

No exemplo acima, imprimimos nosso objeto de soup e ele fornece muito HTML, mas pode ser um pouco difícil de ler. Assim como você pode usar `pprint` no **JSON**, existe algo semelhante para `BeautifulSoup`, uma função chamada `.prettify()`, que usamos no objeto bs4 que queremos tornar mais legíveis, como`soup.prettify()`.

Execute a célula abaixo e veja a diferença.

In [ ]:
# vamos fazer a sopa mais legível
print(soup.prettify())

TPara extrair bits da * sopa *, você precisa conhecer um pouco sobre HTML e CSS.

HTML consiste em tags, agrupadas nos símbolos `<` and `>`. As tags comuns são:

- `a` - a link (a é para **a**nchor - âncora)
- `title` - título da página
- `img` - uma imagem
- `br` - nova linha
- `table` - a tabela
- `tr` - uma linha em uma tabela (tr significa linha da tabela)
- `td` - é usado para definir uma célula padrão na tabela. Significa divisão de tabelas

___

## BeautifulSoup and HTML

Vamos nos concentrar no puro `HTML` nesta sessão.

Quando pedimos `BeautifulSoup` para procurar uma determinada tag, a ferramenta começa na parte superior do nosso HTML e trabalha até o final.

`BeautifulSoup` tem em seu núcleo duas maneiras de procurar o que estamos procurando.
- `.find()` - *retorna o primeiro elemento que conclui a pesquisa (e Nenhum, se não houver nenhum)*
- `.find_all()` - *retorna uma lista de todos os itens que correspondem à pesquisa (e uma lista vazia, se não houver nenhum)*

Portanto, a pergunta que devemos fazer é tipicamente:

- *** Estou procurando uma tag ou estou procurando muitas? ***

![](images/find_vs_find_all.jpg)

___

# .find()

---

Vamos começar procurando por um elemento do nossa `soup` do site de futebol. Vamos usar a função `.find() `.

Para fins de prática, primeiro selecionamos a `HTML` tag `title`, que mostra o título da página.

**Escreva o seguinte** duas linhas de código no campo abaixo. Execute o código e localize a tag title:

title_tag = soup.find('title')

print(title_tag)

Como regra, não estamos interessados no `HTML` em si, mas o que está dentro dele. Podemos extrair isso usando o `.get_text()` método no nosso `title_tag`.

Vamos tentar isso.

**Escreva o seguinte** no campo abaixo e execute novamente:

title = title_tag.get_text()

print(title)

Os títulos HTML vêm em vários tamanhos.

O maior é chamado `h1`, o próximo` h2`, então `h3`, etc.

**Vamos tentar encontrar mais algumas coisas na soup:**

---

- No campo abaixo, extraia um `h4` tag da nossa `soup` com `.find ()`
- Imprimir apenas o conteúdo da tag (ou seja, sem tags)



In [1]:
# use .find() para procurar uma 'h4'-tag

# extraia e imprimima o que está dentro da tag com .get_text()


Muitas vezes vamos usar `.find()` em um elemento, descobrimos usando `find()`. 
Pode ser, por exemplo, se extrairmos uma tabela (`<table>`) do html para um elemento independente e, nesta nova seção, deseja encontrar outro elemento, por exemplo `<tr>`, ou seja, uma linha em uma tabela.

Normalmente, fazemos assim:

```python
table = soup.find('table')
tr = table.find('tr')

```

Dessa forma, é possível cavar cada vez mais uma estrutura HTML - de fato, bastante semelhante à que fizemos ao trabalhar com o **JSON**.

---

![](images/find_all.png)

---

# .find_all()

---

Está bem. Agora sabemos, podemos usar `.find ()` para encontrar uma única tag.

Agora vamos tentar procurar *muitas* tags de algum tipo. Vamos testar a função `.find_all()` , onde encontramos todas as tags de um determinado tipo. Ao contrário de `.find ()`, essa função sempre retorna uma lista - mesmo se houver apenas uma ocorrência ou nenhuma ocorrência.

Podemos pedir ao BeautifulSoup para procurar mais de um tipo de tag, se colocarmos as tags dentro de uma lista.

Então aqui queremos encontrar tudo de <td> ou <th> tags: `.find_all(['th','td'])`
    

***Tente isso:***

In [2]:
# quantas 'h4'-tag você pode encontrar na soup? - dica: use find_all() ao invés de find(). E conte com len()

# quantos links você pode encontrar na soup? dica: procure por 'a' tag

# quantos 'table'-tags você pode encontrar na soup

# em qual das tabelas encontramos os nomes das 50 jogadorea?

# imprime o nome da primeira jogadora


Agora - vamos combinar o que fizemos. Queremos escrever um script, que abra a página, recupere os dados e, no html, encontre todos os nomes dos jogadores e os imprima.

Você pode obtê-lo em duas versões: o código puro sem comentários e um com comentários linha por linha.

In [ ]:
#script sem comentários:

import requests
from bs4 import BeautifulSoup

url = "https://www.worldfootball.net/players_list/frauen-wm-2019-frankreich/nach-mannschaft/1/"

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36'
}

r = requests.get(url, headers=header)
html = r.content.decode('utf-8') 
soup = BeautifulSoup(html, "html.parser")

with open("playernames.txt", 'w', encoding='utf-8') as f:
    table = soup.find_all('table')
    player_table = table[1]
    rows = player_table.find_all('tr')
    for row in rows:
        cols = row.find_all(['th','td'])
        player_name = cols[0].get_text()
        country = cols[2].get_text()
        born = cols[3].get_text()
        height = cols[4].get_text()
        position = cols[5].get_text()
        f.write("{};{};{};{};{}\r\n".format(player_name, country, born, height, position))

In [ ]:
#script com comentários completos para cada etapa:

#nós importamos as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup

#nós definimos a variável url
url = "https://www.worldfootball.net/players_list/frauen-wm-2019-frankreich/nach-mannschaft/1/"

#nós escrevemos um cabeçalho e definimos uma variável
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36'
}

#visitamos a página com nosso cabeçalho definido e obtemos o conteúdo - e adicionamos isso à variável r
r = requests.get(url, headers=header)

#normalmente "html = r.text" seria bom, mas nesse caso, precisamos especificar a codificação para evitar erros
html = r.content.decode('utf-8') 

#executamos o html no BeautifulSoup e usamos o html-parser
soup = BeautifulSoup(html, "html.parser")

#agora definimos um arquivo de texto gravável. Nós chamamos de playernames
with open("playernames.txt", 'w', encoding='utf-8') as f:
    
    #extraímos todas as tabelas para a tabela variável
    table = soup.find_all('table')
    
    #extraímos a tabela com o número de índice 1 e a adicionamos à variável player_table
    player_table = table[1]
    
    #de player_table, encontramos todas as linhas e as adicionamos às linhas variáveis
    rows = player_table.find_all('tr')
    
    #com um "for loop", iteramos pelas linhas.
    for row in rows:
        
        #em cada linha, encontramos todas as tags td (usadas para separar os campos da tabela em cada linha)
        cols = row.find_all(['th','td'])
        
        #o find_all ('td') resulta em uma lista. Para especificar qual tag td, precisamos, usamos números de índice.
        
        #Retiramos o texto da primeira tag td e o adicionamos à variável player_name
        player_name = cols[0].get_text()
        
        #Retiramos o texto da terceira tag td e o adicionamos à variável country
        country = cols[2].get_text()
        
        #Retiramos o texto da quarta tag td e o adicionamos à variável born
        born = cols[3].get_text()
        
        #Retiramos o texto da quinta tag td e o adicionamos à variável height
        height = cols[4].get_text()
        
        #Retiramos o texto da sexta tag td e o adicionamos à variável position
        position = cols[5].get_text()
        
        #Escrevemos uma linha no arquivo de texto. \r\n garante que separamos as linhas corretamente.
        f.write("{};{};{};{};{}\r\n".format(player_name, country, born, height, position))

E finalmente: raspamos a primeira página, mas gostaríamos de raspar todas as 12 páginas.

Nós usamos um `for loop` e criamos os números de 1 a 12 (para inserir no URL) com um `range(1,13)`.

In [ ]:
#nós importamos as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36'
}

with open("playernames.txt", 'w', encoding='utf-8') as f:
    for n in range(1,13):
        url = "https://www.worldfootball.net/players_list/frauen-wm-2019-frankreich/nach-mannschaft/{}/".format(n)
        r = requests.get(url, headers=header)
        html = r.content.decode('utf-8') 
        soup = BeautifulSoup(html, "html.parser")
        table = soup.find_all('table')
        player_table = table[1]
        rows = player_table.find_all('tr')
        for row in rows[1:]:
            cols = row.find_all('td')
            player_name = cols[0].get_text()
            country = cols[2].get_text()
            born = cols[3].get_text()
            height = cols[4].get_text()
            position = cols[5].get_text()
            f.write("{};{};{};{};{}\r\n".format(player_name, country, born, height, position))

# Onde aprender mais

**Tutoriais etc.**

Esperamos que essas três sessões tenham um entendimento básico, mas há muito mais a explorar.

Existem muitos tutoriais na web. Alguns são extremamente bons. Se você tentar um e não gostar ou conseguir, tente outro. Mas os livros também podem ser ótimos.
Aprendi muito com Magnus Lie Hetlands, “Python from Beginning Python: From Novice to Professional”.
https://www.amazon.com/Beginning-Python-Professional-Edition-Professionals/dp/1590599829

Este é o tutorial oficial do Python: https://docs.python.org/2/tutorial/

No Python Wiki: Uma longa lista de tutoriais para não programadores: https://wiki.python.org/moin/BeginnersGuide/NonProgrammers

No Youtube e em muitos outros sites, você encontrará screencasts / gravações de vídeo de programação. Feito da maneira correta, os tutoriais em vídeo podem ser excelentes.


**Se você empacar**

Google para a resposta. Pergunte a um amigo ou grupo de usuários. Leia Como fazer perguntas, antes de postar suas perguntas online. Ele abrange maneiras de ajudar a enquadrar suas perguntas para que outras pessoas possam ajudá-lo da melhor maneira.
https://www.propublica.org/nerds/item/how-to-ask-programming-questions

O Python.org possui listas de grupos de notícias e listas de discussão para iniciantes em Python:
http://mail.python.org/mailman/listinfo

Uma das listas de discussão "Tutor" é descrita assim: "Esta lista é para pessoas que desejam fazer perguntas sobre como aprender programação de computadores com a linguagem Python".

Em resumo, pode-se enviar sua pergunta para a lista, e em breve haverá respostas e essas respostas geralmente são muito competentes. Eles não vão te dar a solução final. Você ainda quer aprender a codificar, certo?

Um grupo de jornalistas que criaram criou a lista de discussão PythonJournos. Funciona da mesma maneira - se você tiver um problema (em relação ao Python), envie-o para a lista de emails e conte com respostas boas e rápidas. No momento (setembro de 19), o grupo tem 498 membros e está aberto a todos.

https://github.com/PythonJournos/LearningPython/wiki
https://groups.google.com/forum/?hl=en#!forum/PythonJournos

Journalism.co.uk: 4 maneiras pelas quais os jornalistas podem obter ajuda com seu código:
https://www.journalism.co.uk/news/4-ways-journalists-can-get-help-with-code/s2/a570167/


Contact: 

**Tommy Kaas** Twitter: @tbkaas mail: tommy.kaas@kaasogmulvad.dk

**Nils Mulvad** Twitter: @nmulvad mail: nils.mulvad@kaasogmulvad.dk



Nota da tradução: No Brasil a Abraji e a Escola de Dados também possuem um fórum para dúvidas e debates sobre Python e outras linguagens: 
        
https://forum.jornalismodedados.org/